In [65]:
#read in relevant packages
import pandas as pd
from PIL import Image
import os
from tqdm import tqdm
import json
import shutil
import random

In [23]:
# Read in the training.csv file
train_df = pd.read_csv("malaria/training.csv")

# Read in the test.csv file
test_df = pd.read_csv("malaria/test.csv")

print(train_df.head())

                     image/checksum  \
0  676bb8e86fc2dbf05dd97d51a64ac0af   
1  1225a18efce159eddf7b0e80e0ea642c   
2  3eaf840523c30fdf38897ffa01e194eb   
3  8a111dffacfa433029492780b9535091   
4  ccef403e971460b86444cca669e68ca1   

                                     image/pathname  image/shape/r  \
0  /images/8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png           1200   
1  /images/10be6380-cbbb-4886-8b9e-ff56b1710576.png           1200   
2  /images/6b14c855-8561-417c-97a4-63fa552842fd.png           1200   
3  /images/13099edb-35d9-438f-b093-2cf2ebf9d255.png           1200   
4  /images/2559636b-f01a-4414-93da-210c3b12d153.png           1200   

   image/shape/c  image/shape/channels  objects/0/bounding_box/minimum/r  \
0           1600                     3                              1057   
1           1600                     3                               734   
2           1600                     3                               724   
3           1600                     3  

/var/folders/qy/7nyf91_x2x519nzr1wfnckz00000gn/T/ipykernel_62784/3582358598.py:2: DtypeWarning: Columns (854,859,864,869,874,879,884,889,894,899,904,909,914,919,924,929,934,939,944,949,954,959,964,969,974,979,984,989,994,999,1004,1009,1014,1019,1024,1029,1034,1039,1044,1049,1054,1059,1064,1069,1074,1079,1084,1089,1094,1099,1104,1109,1114,1119) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv("malaria/training.csv")


In [2]:
# Load the JSON data into a dictionary
with open('malaria/training.json') as f:
    data = json.load(f)

# Create an empty list to hold the data
rows = []

# Iterate through each image object in the JSON data
for img_obj in data:
    img_path = img_obj['image']['pathname']
    img_shape = img_obj['image']['shape']
    
    # Iterate through each object in the current image
    for obj in img_obj['objects']:
        bb = obj['bounding_box']
        rows.append({
            'path': img_path,
            'min_r': bb['minimum']['r'],
            'min_c': bb['minimum']['c'],
            'max_r': bb['maximum']['r'],
            'max_c': bb['maximum']['c'],
            'category': obj['category'],
            'ref': None
        })

# Create a DataFrame from the list of rows
train_file = pd.DataFrame(rows, columns=['path', 'min_r', 'min_c', 'max_r', 'max_c', 'category', 'ref'])


In [3]:
# Load the JSON data into a dictionary
with open('malaria/test.json') as f:
    data = json.load(f)

# Create an empty list to hold the data
rows = []

# Iterate through each image object in the JSON data
for img_obj in data:
    img_path = img_obj['image']['pathname']
    img_shape = img_obj['image']['shape']
    
    # Iterate through each object in the current image
    for obj in img_obj['objects']:
        bb = obj['bounding_box']
        rows.append({
            'path': img_path,
            'min_r': bb['minimum']['r'],
            'min_c': bb['minimum']['c'],
            'max_r': bb['maximum']['r'],
            'max_c': bb['maximum']['c'],
            'category': obj['category'],
            'ref': None
        })

# Create a DataFrame from the list of rows
test_file = pd.DataFrame(rows, columns=['path', 'min_r', 'min_c', 'max_r', 'max_c', 'category', 'ref'])

In [ ]:
train_file['path'] = 'malaria' + train_file['path']
test_file['path'] = 'malaria' + test_file['path']

In [16]:
print(train_file.shape)
train_file.head()

(80052, 7)


,path,min_r,min_c,max_r,max_c,category,ref
0,malaria/images/8d02117d-6c71-4e47-b50a-6cc8d5e...,1057,1440,1158,1540,red blood cell,None
1,malaria/images/8d02117d-6c71-4e47-b50a-6cc8d5e...,868,1303,971,1403,red blood cell,None
2,malaria/images/8d02117d-6c71-4e47-b50a-6cc8d5e...,578,900,689,1008,red blood cell,None
3,malaria/images/8d02117d-6c71-4e47-b50a-6cc8d5e...,304,611,408,713,red blood cell,None
4,malaria/images/8d02117d-6c71-4e47-b50a-6cc8d5e...,198,881,312,1003,red blood cell,None


In [9]:
print(test_file.shape)
train_file.head()

(5922, 7)


,path,min_r,min_c,max_r,max_c,category,ref
0,malaria/images/8d02117d-6c71-4e47-b50a-6cc8d5e...,1057,1440,1158,1540,red blood cell,None
1,malaria/images/8d02117d-6c71-4e47-b50a-6cc8d5e...,868,1303,971,1403,red blood cell,None
2,malaria/images/8d02117d-6c71-4e47-b50a-6cc8d5e...,578,900,689,1008,red blood cell,None
3,malaria/images/8d02117d-6c71-4e47-b50a-6cc8d5e...,304,611,408,713,red blood cell,None
4,malaria/images/8d02117d-6c71-4e47-b50a-6cc8d5e...,198,881,312,1003,red blood cell,None


In [13]:
#check if all paths have an image to it
# Iterate over the rows of the DataFrame
for index, row in train_file.iterrows():
    image_path = row['path']
    if not os.path.isfile(image_path):
        train_file.drop(index, inplace=True)

# Reset the index of the DataFrame
train_file.reset_index(drop=True, inplace=True)

In [15]:
len(train_file)

80052

In [5]:
train_len = len(train_file['path'].unique())
print('Total number of Images in train dataset:', train_len)
test_len = len(test_file['path'].unique())
print('Total number of Images in test dataset :', test_len)

Total number of Images in train dataset: 1208
Total number of Images in test dataset : 120


In [22]:
train_counts = train_file['category'].value_counts()
test_counts = test_file['category'].value_counts()

# print counts for each category in train and test datasets
print('Train dataset:')
print(train_counts)
print('\nTest dataset:')
print(test_counts)
print('\nTotal dataset:')
print(train_counts + test_counts)

Train dataset:
red blood cell    77362
trophozoite        1472
difficult           439
ring                353
schizont            179
gametocyte          144
leukocyte           103
Name: category, dtype: int64

Test dataset:
red blood cell    5614
ring               169
trophozoite        111
gametocyte          12
schizont            11
difficult            5
Name: category, dtype: int64

Total dataset:
difficult           444.0
gametocyte          156.0
leukocyte             NaN
red blood cell    82976.0
ring                522.0
schizont            190.0
trophozoite        1583.0
Name: category, dtype: float64


In [ ]:
print(len(train_file))

In [18]:
# cropping TRAIN images
# create a new directory to store the cropped images
if not os.path.exists('cropped_images'):
    os.makedirs('cropped_images')

# get the total number of rows in train_file dataframe
total_rows = len(train_file)

# iterate over rows in train_file dataframe
for index, row in tqdm(train_file.iterrows(), total=total_rows):
    # open image from path column
    img = Image.open(row['path'])
    # crop image based on coordinates
    cropped_img = img.crop((row['min_c'], row['min_r'], row['max_c'], row['max_r']))
    # save cropped image with new file name in the cropped_images folder
    filename = os.path.join('cropped_images', f"{os.path.splitext(os.path.basename(row['path']))[0]}_{str(row['category'])}_{str(index)}.png")
    cropped_img.save(filename)

100%|███████████████████████████████████| 80052/80052 [1:10:48<00:00, 18.84it/s]


In [19]:
# cropping TEST images
# create a new directory to store the cropped images
if not os.path.exists('cropped_images'):
    os.makedirs('cropped_images')

# get the total number of rows in train_file dataframe
total_rows = len(test_file)

# iterate over rows in train_file dataframe
for index, row in tqdm(test_file.iterrows(), total=total_rows):
    # open image from path column
    img = Image.open(row['path'])
    # crop image based on coordinates
    cropped_img = img.crop((row['min_c'], row['min_r'], row['max_c'], row['max_r']))
    # save cropped image with new file name in the cropped_images folder
    filename = os.path.join('cropped_images', f"{os.path.splitext(os.path.basename(row['path']))[0]}_{str(row['category'])}_{str(index)}.png")
    cropped_img.save(filename)

100%|███████████████████████████████████████| 5922/5922 [02:23<00:00, 41.40it/s]


In [20]:
# Filtering out of all cropped images
src_dir = "cropped_images"
dest_dir = "filtered_cropped_images"
os.makedirs(dest_dir, exist_ok=True)

all_files = os.listdir(src_dir)
# Filter for 'red blood cell' and limit to 5000
rbc_files = [f for f in all_files if "red blood cell" in f]
random.shuffle(rbc_files)
rbc_files = rbc_files[:5000]

# Filter for the other categories
categories = ["trophozoite", "ring", "schizont", "gametocyte"]
category_files = {cat: [f for f in all_files if cat in f] for cat in categories}

# Copy files
for f in rbc_files:
    shutil.copy(os.path.join(src_dir, f), dest_dir)

for cat, files in category_files.items():
    for f in files:
        shutil.copy(os.path.join(src_dir, f), dest_dir)

# Generate and print summary table
summary = {cat: len(files) for cat, files in category_files.items()}
summary["red blood cell"] = len(rbc_files)
df_crop = pd.DataFrame.from_dict(summary, orient='index', columns=['Count'])
print(df_crop)


                Count
trophozoite      1583
ring              522
schizont          190
gametocyte        156
red blood cell   5000


In [25]:
def crop_images(df, image_dir, cropped_dir):
    # create a new directory to store the cropped images
    if not os.path.exists(cropped_dir):
        os.makedirs(cropped_dir)

    # get the total number of rows in dataframe
    total_rows = len(df)

    # iterate over rows in dataframe
    for index, row in tqdm(df.iterrows(), total=total_rows):
        # open image from path column
        img = Image.open(os.path.join(image_dir, row['path']))
        # crop image based on coordinates
        cropped_img = img.crop((row['min_c'], row['min_r'], row['max_c'], row['max_r']))
        # save cropped image with new file name in the cropped_images folder
        filename = os.path.join(cropped_dir, f"{os.path.splitext(os.path.basename(row['path']))[0]}_{str(row['category'])}_{str(index)}.png")
        cropped_img.save(filename)

In [29]:
# select a random image to crop
selected_image = random.choice(train_file['path'].unique())

# create a new dataframe that only contains the rows related to the selected image
df_selected = train_file[train_file['path'] == selected_image]

# crop the images
crop_images(df_selected, '', 'crop_test')

100%|█████████████████████████████████████████| 198/198 [00:11<00:00, 16.83it/s]
